<a href="https://colab.research.google.com/github/pavansai26/RFM-ANALYSIS-FOR-ONLINE-STORE-DATA/blob/main/RFM_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
data=pd.read_csv('/gdrive/My Drive/Colab Notebooks/Data_csv.csv')

In [ ]:
Rtl_data = data

In [ ]:
Rtl_data.head()
#Import Online Retail Data containing transactions from 01/12/2010 and 09/12/2011

In [ ]:
#Check the shape (number of columns and rows) in the dataset
Rtl_data.shape

In [ ]:
Rtl_data.columns

In [ ]:
#Customer distribution by country
country_cust_data=Rtl_data[['transaction country','customer id']].drop_duplicates()
country_cust_data.groupby(['transaction country'])['customer id'].aggregate('count').reset_index().sort_values('customer id', ascending=False)

In [ ]:
#Keep only United Kingdom data
Rtl_data = Rtl_data[( Rtl_data["transaction country"]=='United Kingdom')].reset_index(drop=True)

In [ ]:
Rtl_data

In [ ]:
#Check for missing values in the dataset
Rtl_data.isnull().sum(axis=0)

In [ ]:
#Remove missing values from CustomerID column, can ignore missing values in description column
Rtl_data = Rtl_data[pd.notnull(Rtl_data['customer id'])]



In [ ]:
#Validate if there are any negative values in Quantity column
Rtl_data['quantity sold'].min()

In [ ]:
#Validate if there are any negative values in UnitPrice column
Rtl_data['unit price'].min()

In [ ]:
#Filter out records with negative values
Rtl_data = Rtl_data[(Rtl_data['quantity sold']>0)]

In [ ]:
#Convert the string date field to datetime
Rtl_data['InvoiceDate'] = pd.to_datetime(Rtl_data['transaction timestamp'])

In [ ]:
#Add new column depicting total amount
Rtl_data['TotalAmount'] = Rtl_data['quantity sold'] * Rtl_data['unit price']

In [ ]:
#Check the shape (number of columns and rows) in the dataset after data is cleaned
Rtl_data.shape

In [ ]:
Rtl_data.head()

# **Recency = Latest Date - Last Inovice Data**

# **Frequency = count of invoice no. of transaction(s)**

# **Monetary = Sum of Total Amount for each customer**

In [ ]:
import datetime as dt

In [ ]:
#Set Latest date 2011-12-10 as last invoice date was 2011-12-09. This is to calculate the number of days from recent purchase
Latest_Date = dt.datetime(2011,12,10)

In [ ]:
#Create RFM Modelling scores for each customer
RFMScores = Rtl_data.groupby('customer id').agg({'InvoiceDate': lambda x: (Latest_Date - x.max()).days, 'transaction id': lambda x: len(x), 'TotalAmount': lambda x: x.sum()})

In [ ]:
#Convert Invoice Date into type int
RFMScores['InvoiceDate'] = RFMScores['InvoiceDate'].astype(int)


In [ ]:
#Rename column names to Recency, Frequency and Monetary
RFMScores.rename(columns={'InvoiceDate': 'Recency', 
                         'transaction id': 'Frequency', 
                         'TotalAmount': 'Monetary'}, inplace=True)

RFMScores.reset_index().head()

In [ ]:
#Descriptive Statistics (Recency)
RFMScores.Recency.describe()

In [ ]:
#Recency distribution plot
import seaborn as sns
x = RFMScores['Recency']
ax = sns.distplot(x)

In [ ]:
#Descriptive Statistics (Frequency)
RFMScores.Frequency.describe()

In [ ]:
#Frequency distribution plot, taking observations which have frequency less than 1000
import seaborn as sns
x = RFMScores.query('Frequency < 1000')['Frequency']
ax = sns.distplot(x)

In [ ]:
#Descriptive Statistics (Monetary)
RFMScores.Monetary.describe()

In [ ]:
#Monateray distribution plot, taking observations which have monetary value less than 10000
import seaborn as sns
x = RFMScores.query('Monetary < 10000')['Monetary']
ax = sns.distplot(x)

In [ ]:
#Split into four segments using quantiles
quantiles = RFMScores.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [ ]:
quantiles

In [ ]:
#Functions to create R, F and M segments
def RScoring(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FnMScoring(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [ ]:
#Calculate Add R, F and M segment value columns in the existing dataset to show R, F and M segment values
RFMScores['R'] = RFMScores['Recency'].apply(RScoring, args=('Recency',quantiles,))
RFMScores['F'] = RFMScores['Frequency'].apply(FnMScoring, args=('Frequency',quantiles,))
RFMScores['M'] = RFMScores['Monetary'].apply(FnMScoring, args=('Monetary',quantiles,))
RFMScores.head()

In [ ]:
#Calculate and Add RFMGroup value column showing combined concatenated score of RFM
RFMScores['RFMGroup'] = RFMScores.R.map(str) + RFMScores.F.map(str) + RFMScores.M.map(str)

#Calculate and Add RFMScore value column showing total sum of RFMGroup values
RFMScores['RFMScore'] = RFMScores[['R', 'F', 'M']].sum(axis = 1)
RFMScores.head()

In [ ]:
#Assign Loyalty Level to each customer
Loyalty_Level = ['Platinum', 'Gold', 'Silver', 'Bronze']
Score_cuts = pd.qcut(RFMScores.RFMScore, q = 4, labels = Loyalty_Level)
RFMScores['RFM_Loyalty_Level'] = Score_cuts.values
RFMScores.reset_index().head()

In [ ]:
#Validate the data for RFMGroup = 111
RFMScores[RFMScores['RFMGroup']=='111'].sort_values('Monetary', ascending=False).reset_index()

In [ ]:
#Validate the data for RFMGroup = 444
RFMScores[RFMScores['RFMGroup']=='444'].sort_values('Monetary', ascending=False).reset_index()